In [ ]:
#Để lập trình hàm softmax, chúng ta cần thực hiện các bước sau:

#Tính tử số: Với mỗi phần tử z_i trong vector z, tính giá trị exp(z_i).
#Tính mẫu số: Tính tổng của tất cả các giá trị exp(z_i) đã tính ở bước 1.
#Tính giá trị softmax: Với mỗi phần tử z_i, chia giá trị exp(z_i) cho mẫu số đã tính ở bước 2. Kết quả sẽ là giá trị softmax tương ứng với phần tử z_i đó.

In [1]:
import numpy as np

def softmax(z):
  """
  Hàm softmax cho vector đầu vào z.

  Args:
    z: Vector đầu vào z ∈ Rd.

  Returns:
    Vector a là kết quả của hàm softmax, với các phần tử nằm trong khoảng (0, 1) và có tổng bằng 1.
  """

  # Tính tử số.
  exp_z = np.exp(z)

  # Tính mẫu số.
  sum_exp_z = np.sum(exp_z)

  # Tính giá trị softmax.
  a = exp_z / sum_exp_z

  return a

In [4]:
z = np.array([1, 2, 3])
a = softmax(z)
print(a)
print("Tong cua cac phan tu:",np.sum(a))

[0.09003057 0.24472847 0.66524096]
Tong cua cac phan tu: 1.0


In [5]:
import numpy as np

def softmax_stable(z):
  """
  Hàm softmax với kỹ thuật chống tràn số.

  Args:
    z: Mảng một chiều đầu vào z (score vector).

  Returns:
    Mảng một chiều chứa các giá trị a_i theo công thức softmax_stable.
  """
  c = np.max(z)
  z_stable = z - c
  exp_z = np.exp(z_stable)
  sum_exp_z = np.sum(exp_z)
  a = exp_z / sum_exp_z
  return a

In [6]:
z = np.array([1000, 1001, 1002])  # Các giá trị lớn

a_stable = softmax_stable(z)
a_normal = softmax(z)

print("Kết quả softmax_stable:", a_stable)
print("Kết quả softmax thường:", a_normal)

Kết quả softmax_stable: [0.09003057 0.24472847 0.66524096]
Kết quả softmax thường: [nan nan nan]


C:\Users\Admin\AppData\Local\Temp\ipykernel_6696\655182213.py:15: RuntimeWarning: overflow encountered in exp
  exp_z = np.exp(z)
C:\Users\Admin\AppData\Local\Temp\ipykernel_6696\655182213.py:21: RuntimeWarning: invalid value encountered in divide
  a = exp_z / sum_exp_z


In [7]:
import numpy as np

def softmax_matrix_stable(Z):
  """
  Calculates the stable softmax for each row of a matrix.

  Args:
    Z: A 2D numpy array (matrix) where each row represents scores for a data point.

  Returns:
    A 2D numpy array (matrix) A of the same shape as Z, where each row contains 
    the softmax probabilities for the corresponding row in Z.
  """
  c = np.max(Z, axis=1, keepdims=True)  # Get max for each row, keepdims for broadcasting
  exp_Z = np.exp(Z - c)              # Subtract max and exponentiate
  sum_exp_Z = np.sum(exp_Z, axis=1, keepdims=True)  # Sum for each row
  return exp_Z / sum_exp_Z            # Normalize to get softmax probabilities

In [8]:
Z = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

A = softmax_matrix_stable(Z)
print(A)

[[0.09003057 0.24472847 0.66524096]
 [0.09003057 0.24472847 0.66524096]
 [0.09003057 0.24472847 0.66524096]]


In [10]:
import numpy as np

def softmax_matrix_stable(Z):
  c = np.max(Z, axis=1, keepdims=True); return np.exp(Z - c) / np.sum(np.exp(Z - c), axis=1, keepdims=True)

Z = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
A = softmax_matrix_stable(Z)
print(A)

[[0.09003057 0.24472847 0.66524096]
 [0.09003057 0.24472847 0.66524096]
 [0.09003057 0.24472847 0.66524096]]


In [ ]:
#https://how.dev/answers/implementation-of-relu-in-pytorch
#BT: https://www.kaggle.com/code/hafizwaqas101/pytorch-with-relu-sigmoid-tah/notebook

PyTorch with ReLU, Sigmoid, Tah

In [ ]:
# Import các thư viện cần thiết
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Đặt seed cho tính tái lập
torch.manual_seed(2)

In [ ]:
# ===============================
# 1. Chuẩn bị dữ liệu
# ===============================

# Thực hiện các phép biến đổi ảnh
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Tải dataset MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transform, 
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transform, 
                                          download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=64, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=64, 
                                          shuffle=False)


In [ ]:
# ===============================
# 2. Xây dựng Model Neural Network
# ===============================

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)  # Input layer
        self.fc2 = nn.Linear(128, 64)     # Hidden layer
        self.fc3 = nn.Linear(64, 10)      # Output layer
    
    def forward(self, x):
        x = x.view(-1, 28*28)             # Flatten ảnh 28x28 thành vector 784
        x = F.relu(self.fc1(x))           # Activation ReLU
        x = torch.sigmoid(self.fc2(x))    # Activation Sigmoid
        x = torch.tanh(self.fc3(x))       # Activation Tanh
        return x

# Khởi tạo model
model = NeuralNet()


In [ ]:

# ===============================
# 3. Loss và Optimizer
# ===============================

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:

# ===============================
# 4. Huấn luyện Model
# ===============================

num_epochs = 5
train_loss_list = []

for epoch in range(num_epochs):
    epoch_loss = 0
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    average_loss = epoch_loss / len(train_loader)
    train_loss_list.append(average_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}')

In [ ]:
# ===============================
# 5. Đánh giá Model
# ===============================

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'\nAccuracy of the model on the test images: {accuracy:.2f}%')

In [ ]:
# ===============================
# 6. Hiển thị Loss
# ===============================

plt.plot(train_loss_list, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ===============================
# 7. Hiển thị Một Số Hình Ảnh Mẫu
# ===============================

examples = iter(test_loader)
example_data, example_targets = examples.next()

for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.imshow(example_data[i][0], cmap='gray')
    plt.title(f'Label: {example_targets[i]}')
    plt.axis('off')
plt.show()

In [ ]:
# ===============================
# 8. Lưu Model
# ===============================
torch.save(model.state_dict(), 'mnist_model.pth')
print("\nModel đã được lưu thành công dưới tên 'mnist_model.pth'.")


PyTorch Practice 10

In [ ]:
# Code Modules, Classes & Functions
import numpy as np,pandas as pd,pylab as pl
import h5py,torch
from tensorflow import image as timage
from torchvision import transforms,utils
from torch.utils.data import DataLoader as tdl
from torch.utils.data import Dataset as tds
import torch.nn as tnn
from IPython.core.magic import register_line_magic
dev=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class TData(tds):
    def __init__(self,X,y):   
        self.X=torch.tensor(X,dtype=torch.float32)
        self.y=torch.tensor(y,dtype=torch.int32)
    def __getitem__(self,index):
        train_img,train_lbl=self.X[index],self.y[index]
        return train_img,train_lbl
    def __len__(self):
        return self.y.shape[0]

In [ ]:
class VGG19(tnn.Module):
    def __init__(self,num_classes):
        super(VGG19,self).__init__()    
        self.block1=tnn.Sequential(
            tnn.Conv2d(in_channels=3,out_channels=64,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1), 
            tnn.ReLU(),
            tnn.Conv2d(in_channels=64,out_channels=64,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),
            tnn.MaxPool2d(kernel_size=(2,2),stride=(2,2)) )        
        self.block2=tnn.Sequential(
            tnn.Conv2d(in_channels=64,out_channels=128,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),
            tnn.Conv2d(in_channels=128,out_channels=128,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),
            tnn.MaxPool2d(kernel_size=(2,2),stride=(2,2)) )      
        self.block3=tnn.Sequential(
            tnn.Conv2d(in_channels=128,out_channels=256,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),
            tnn.Conv2d(in_channels=256,out_channels=256,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),        
            tnn.Conv2d(in_channels=256,out_channels=256,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),
            tnn.Conv2d(in_channels=256,out_channels=256,
                       kernel_size=(3,3),stride=(1,1),
                      padding=1),
            tnn.ReLU(),
            tnn.MaxPool2d(kernel_size=(2,2),stride=(2,2)) )         
        self.block4=tnn.Sequential(   
            tnn.Conv2d(in_channels=256,out_channels=512,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),        
            tnn.Conv2d(in_channels=512,out_channels=512,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),        
            tnn.Conv2d(in_channels=512,out_channels=512,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),
            tnn.Conv2d(in_channels=512,out_channels=512,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),   
            tnn.MaxPool2d(kernel_size=(2,2),stride=(2,2)) )       
        self.block5=tnn.Sequential(
            tnn.Conv2d(in_channels=512,out_channels=512,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),            
            tnn.Conv2d(in_channels=512,out_channels=512,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),            
            tnn.Conv2d(in_channels=512,out_channels=512,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),
            tnn.Conv2d(in_channels=512,out_channels=512,
                       kernel_size=(3,3),stride=(1,1),
                       padding=1),
            tnn.ReLU(),   
            tnn.MaxPool2d(kernel_size=(2,2),stride=(2,2)) )        
        self.classifier=tnn.Sequential(
            tnn.Linear(512,4096),
            tnn.ReLU(True),
            tnn.Linear(4096,4096),
            tnn.ReLU(True),
            tnn.Linear(4096,num_classes) )            
        for m in self.modules():
            if isinstance(m,torch.nn.Conv2d):
                m.weight.detach().normal_(0,.05)
                if m.bias is not None:
                    m.bias.detach().zero_()
            elif isinstance(m, torch.nn.Linear):
                m.weight.detach().normal_(0,.05)
                m.bias.detach().detach().zero_()      
    def forward(self,x):
        x=self.block1(x)
        x=self.block2(x)
        x=self.block3(x)
        x=self.block4(x)
        x=self.block5(x)
        logits=self.classifier(x.view(-1,512))
        probs=tnn.functional.softmax(logits,dim=1)
        return logits,probs

In [ ]:
pixels=48
fpath='../input/tf-cats-vs-dogs/'
f='CatDogImages.h5'
f=h5py.File(fpath+f,'r')
keys=list(f.keys()); print(keys)
x_test=np.array(f[keys[0]])
y_test=np.array(f[keys[1]]).reshape(-1)
x_train=np.array(f[keys[2]])
y_train=np.array(f[keys[3]]).reshape(-1)
[[x_train,x_valid,x_test],
 [y_train,y_valid,y_test]]=\
resize_display(x_train,y_train,x_test,y_test,pixels)

random_seed=1; batch_size=128;
train=TData(x_train,y_train)
valid=TData(x_valid,y_valid)
test=TData(x_test,y_test)
train_loader=tdl(dataset=train,
                  batch_size=batch_size,shuffle=True)
valid_loader=tdl(dataset=valid,
                  batch_size=batch_size,shuffle=True)
test_loader=tdl(dataset=test,
                 batch_size=batch_size,shuffle=False)
display_examples(valid_loader,pixels)

In [ ]:
torch.manual_seed(random_seed)
num_classes=2; learning_rate=.001
model=VGG19(num_classes); model.to(dev)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
%train_run 5

In [ ]:
%print_acc 1

In [ ]:
import numpy as np
import pandas as pd
def soChiaHetCho5(n):
    if ( n % 5 == 0):
        return True
    else: 
        return False

def sapXepMang(array):
    return np.sort(array)

arraySample = [4, 5, 6, 22, 55, 99, 15, 35, 11]

def timSoChiaHetCho5(arrr):
    arr = arrr
    arrChiaHetCho5 = []
    for i in arr:
        if (soChiaHetCho5(i) == True):
            arrChiaHetCho5.append(i)
    print("Nhung so chia het cho 5 trong mang la: ", arrChiaHetCho5)    

    
arrayChiaHetCho5 = timSoChiaHetCho5(sapXepMang(arraySample))


def daoNguocMangThuCong(arr):
    return np.flip(arr)


print("Test dao nguoc mang: ", daoNguocMangThuCong(arrayChiaHetCho5))


tesstRandom = np.random.random((1))
print("test random: ", tesstRandom)






Nhung so chia het cho 5 trong mang la:  [5, 15, 35, 55]
Test dao nguoc mang:  None
test random:  [0.66536603]


In [88]:
def vongQuayMayMan(conSoMayMan):
    arr = np.random.randint(1, 100, 10)
    print("Xo so hom nay: ", arr)

    nguoiMayMan = conSoMayMan
    for i in range(0, len(arr)):
        if (arr[i] == nguoiMayMan):
            print("Chuc mung ban da trung so roi!")
            print("Con so may man cua ban la", nguoiMayMan)


def kiemSoatTreEm(agee, name):
    age = 18
    agePerson = agee
    if(agePerson < age):
        print("Xin chao", name, ", rat xin loi nhung ban chua du tuoi!")
    else:
        print("OK, chuc mung ", name)

 
print("===============================")
def person():
    age = int(input("Nhap do tuoi cua ban: "))
    name = input("Nhap ten cua ban: ")
    conSoMayMan = int(input("Nhap con so may man cua ban: "))
    kiemSoatTreEm(age, name)
    vongQuayMayMan(conSoMayMan)

# print("Kiem tra person co du tuoi khong?")
# person()



def kiemTraSucKhoeNghiaVu(name, age, heigh, weigh):
    muc1 = True
    muc2 = True
    muc3 = True 
    if (age < 18):
        muc1 = False
    if (heigh < 140 or weigh > 100):
        muc2 = muc3 = False
    if (muc1 == True & muc2 == True & muc3 == True):
        print("Chuc mung", name, "da du dieu kien nhap ngu")
    else:
        print(name,"chua du dieu kien nhap ngu, cam on da den day")



kiemTraSucKhoeNghiaVu("Nghia", 19, 180, 50)




Chuc mung Nghia da du dieu kien nhap ngu
